I have a version of this file that worked to train an sbert model here:
https://github.com/fflory/train_sbert.git

In [1]:
# import torch
# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# tokenizer = DistilBertTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
# model = DistilBertForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_class_id = logits.argmax().item()
# model.config.id2label[predicted_class_id]


Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install transformers datasets evaluate accelerate scikit-learn tensorflow
```

In [2]:
from huggingface_hub import notebook_login

/Users/Felix.Flory/git/train_sbert/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from huggingface_hub import login

# Set your Hugging Face token as an environment variable
# os.environ["HUGGINGFACE_HUB_TOKEN"] = "your-token-here"

# Login using the token from the environment variable
login(token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))

In [4]:
from datasets import load_dataset

imdb = load_dataset("imdb")

In [5]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")


In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [8]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

In [9]:
from transformers import DataCollatorWithPadding

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

In [12]:
import numpy as np

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="/Users/Felix.Flory/git/train_sbert/my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

 16%|█▌        | 500/3126 [05:33<29:36,  1.48it/s]

{'loss': 0.3221, 'grad_norm': 11.273344039916992, 'learning_rate': 1.6801023672424827e-05, 'epoch': 0.32}


 32%|███▏      | 1000/3126 [11:18<25:25,  1.39it/s]

{'loss': 0.2493, 'grad_norm': 4.94019889831543, 'learning_rate': 1.3602047344849649e-05, 'epoch': 0.64}


 48%|████▊     | 1500/3126 [17:04<18:47,  1.44it/s]

...

                                                   
100%|██████████| 3126/3126 [53:56<00:00,  1.79it/s]

{'eval_loss': 0.23349902033805847, 'eval_accuracy': 0.93084, 'eval_runtime': 708.4947, 'eval_samples_per_second': 35.286, 'eval_steps_per_second': 2.206, 'epoch': 2.0}


100%|██████████| 3126/3126 [53:58<00:00,  1.04s/it]
No files have been modified since last commit. Skipping to prevent empty commit.


{'train_runtime': 3238.3049, 'train_samples_per_second': 15.44, 'train_steps_per_second': 0.965, 'train_loss': 0.2056924858996293, 'epoch': 2.0}


TrainOutput(global_step=3126, training_loss=0.2056924858996293, metrics={'train_runtime': 3238.3049, 'train_samples_per_second': 15.44, 'train_steps_per_second': 0.965, 'total_flos': 6556904415524352.0, 'train_loss': 0.2056924858996293, 'epoch': 2.0})

In [17]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/fflory/my_awesome_model/commit/598f8283da0856753ec2e9817b167bd5cce5166a', commit_message='End of training', commit_description='', oid='598f8283da0856753ec2e9817b167bd5cce5166a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/fflory/my_awesome_model', endpoint='https://huggingface.co', repo_type='model', repo_id='fflory/my_awesome_model'), pr_revision=None, pr_num=None)

In [20]:
# from transformers import create_optimizer
# import tensorflow as tf
# batch_size = 16
# num_epochs = 5
# batches_per_epoch = len(tokenized_imdb["train"]) // batch_size
# total_train_steps = int(batches_per_epoch * num_epochs)
# optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [21]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [23]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="fflory/my_awesome_model")
classifier(text)
[{'label': 'POSITIVE', 'score': 0.9994940757751465}]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9994940757751465}]